# Resume Parsing

## 1. Load data

In [1]:
import pandas as pd
import numpy as np

df_resume = pd.read_csv("data/resume.csv")

In [41]:
# df_resume.Category.unique()

In [3]:
# df_resume.shape

In [4]:
df_resume = df_resume.reindex(np.random.permutation(df_resume.index))
# df_resume = df_resume.copy().iloc[:1000, ]
# df_resume.shape

## 2. Load skill data


In [5]:
import spacy

nlp = spacy.load('en_core_web_md')
skill_path = 'data/skills.jsonl'

In [6]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_path)
# nlp.pipe_names

In [42]:
# doc = nlp("Chaky loves ajax.")
# doc.ents

## 3. extract skills from this resume.csv

In [43]:
# df_resume.head()

In [9]:
#clean our data
from spacy.lang.en.stop_words import STOP_WORDS

def preprocessing(sentence):
    stopwords    = list(STOP_WORDS)
    doc          = nlp(sentence)
    clean_tokens = []
    
    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SYM' and \
            token.pos_ != 'SPACE':
                clean_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(clean_tokens)

In [12]:
for i, row in df_resume.iterrows():
    clean_text = preprocessing(row.Resume_str)
    df_resume.at[i, 'Clean_resume'] = clean_text

In [44]:
# df_resume.head()

## 4. Let's really extract skills!!

In [14]:
def get_skills(text):
    
    doc = nlp(text)
    
    skills = []
    
    for ent in doc.ents:
        if ent.label_ == 'SKILL':
            skills.append(ent.text)
            
    return skills

def unique_skills(x):
    return list(set(x))

In [16]:
df_resume['Skills'] = df_resume.Clean_resume.apply(get_skills)
df_resume['Skills'] = df_resume.Skills.apply(unique_skills)

In [45]:
# df_resume.Skills.iloc[0]

## Task1,2 Additional feature

I created lookfor_skill function, the purpose of the function is to find candidates who were qualified for specific skill that HR look for. For example, if HR look for candidate who have project management skill so they can specify the skill and it will display candidates who have project management skill.

In [20]:
def lookfor_skill(skill):
    qualified_id = []
    for i in range(len(df_resume)):
        if skill in (df_resume.Skills.iloc[i]):
            qualified_id.append(df_resume.ID.iloc[i])
        else:
            # return f"{df_resume.ID.iloc[i]} is not qualified"
            continue
    return qualified_id

In [21]:
qualified_person = df_resume.Clean_resume.apply(lambda x: lookfor_skill("project management"))[0]

In [24]:
# qualified_person

## 7. Let's load the PDF - add some realism

In [35]:
from PyPDF2 import PdfReader

reader = PdfReader("data/chaklam_resume.pdf")
page = reader.pages[0]
text = page.extract_text()

In [36]:
text = preprocessing(text)

In [37]:
doc = nlp(text)

In [40]:
get_skills(doc)

['mobile',
 'python',
 'java',
 'pytorch',
 'deep learning',
 'natural language processing',
 'software engineering',
 'testing',
 'data structures',
 'algorithms',
 'ai',
 'question answering',
 'language model',
 'deep learning',
 'language model',
 'engineering',
 'business',
 'engineering',
 'computer science',
 'engineering',
 'computer science',
 'localization',
 'framework',
 'design',
 'interaction']

In [ ]:
# colors = {"SKILL": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
# options = {"colors": colors}

# displacy.render(doc, style='ent', options=options)